In [1]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.window import Window
from sklearn import preprocessing # https://github.com/Snowflake-Labs/snowpark-python-demos/tree/main/sp4py_utilities
from snowflake.snowpark.functions import col
import snowflake
import snowflake.snowpark

import getpass
import pandas as pd
import matplotlib.pyplot as plt
import decimal
import joblib 
from datetime import datetime, timedelta
import random

In [2]:
accountname = "HIIOYKL-IX77996" # ORGNAME-ACCOUNTNAME (separated by minus sign)
username =  "Nathan" # SNOWFLAKE-USERNAME
password = "Nathan5!"# SNOWFLAKE-PASSWORD

In [3]:
connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "ACCOUNTADMIN",
    "database": "FROSTBYTE_TASTY_BYTES",

    "warehouse": "COMPUTE_WH"
}

session = Session.builder.configs(connection_parameters).create()

## Generating Data

In [4]:
truck_cols=['TRUCK_ID','MENU_TYPE_GYROS_ENCODED', 'MENU_TYPE_CREPES_ENCODED',
       'MENU_TYPE_BBQ_ENCODED', 'MENU_TYPE_SANDWICHES_ENCODED',
       'MENU_TYPE_Mac & Cheese_encoded', 'MENU_TYPE_POUTINE_ENCODED',
       'MENU_TYPE_ETHIOPIAN_ENCODED', 'MENU_TYPE_TACOS_ENCODED',
       'MENU_TYPE_Ice Cream_encoded', 'MENU_TYPE_Hot Dogs_encoded',
       'MENU_TYPE_CHINESE_ENCODED', 'MENU_TYPE_Grilled Cheese_encoded',
       'MENU_TYPE_VEGETARIAN_ENCODED', 'MENU_TYPE_INDIAN_ENCODED',
       'MENU_TYPE_RAMEN_ENCODED']
location_cols=[ 'CITY_SEATTLE_ENCODED',
       'CITY_DENVER_ENCODED', 'CITY_San Mateo_encoded',
       'CITY_New York City_encoded', 'CITY_BOSTON_ENCODED',
       'REGION_NY_ENCODED', 'REGION_MA_ENCODED', 'REGION_CO_ENCODED',
       'REGION_WA_ENCODED', 'REGION_CA_ENCODED', 'LAT', 'LONG', 'LOCATION_ID']

In [5]:
date = '2021-8-23'
datetime_object = datetime.strptime(date, '%Y-%m-%d')

In [6]:
def generate_date_data(df,datetime_object):
    df['date'] = pd.to_datetime(datetime_object)
    df['MONTH'] = df['date'].dt.month
    df['DOW'] = df['date'].dt.weekday
    df['DAY'] = df['date'].dt.day
    df['WOM'] = (df['DAY'] - 1) // 7 + 1
    df['YEAR'] = df['date'].dt.year

    public_holidays = [
     {'Month': 7, 'Day': 4, 'DOW': None, 'WOM': None},  # 4th of July
   {'Month': 12, 'Day': 24, 'DOW': None, 'WOM': None},  # Christmas Eve
    {'Month': 12, 'Day': 25, 'DOW': None, 'WOM': None},  # Christmas Day
    {'Month': 10, 'Day': None, 'DOW': '0', 'WOM': 2},  # Columbus Day (second Monday in October)
    {'Month': 6, 'Day': 19, 'DOW': None, 'WOM': None},  # Juneteenth
    {'Month': 9, 'Day': None, 'DOW': '0', 'WOM': 1},  # Labor Day (first Monday in September)
    {'Month': 1, 'Day': None, 'DOW': '0', 'WOM': 3},  # Martin Luther King, Jr. Day (third Monday in January)
    {'Month': 5, 'Day': None, 'DOW': '0', 'WOM': -1},  # Memorial Day (last Monday in May)
    {'Month': 1, 'Day': 1, 'DOW': None, 'WOM': None},  # New Year's Day
    {'Month': 12, 'Day': 31, 'DOW': None, 'WOM': None},  # New Year's Eve
    {'Month': 11, 'Day': None, 'DOW': '3', 'WOM': 4},  # Thanksgiving Day (fourth Thursday in November)
    {'Month': 11, 'Day': None, 'DOW': '2', 'WOM': 4},  # Thanksgiving Eve (fourth Wednesday in November)
    {'Month': 2, 'Day': 14, 'DOW': None, 'WOM': None},  # Valentine's Day
    {'Month': 11, 'Day': 11, 'DOW': None, 'WOM': None},  # Veterans Day
    {'Month': 10, 'Day': 31, 'DOW': None, 'WOM': None},  # Halloween
    {'Month': 3, 'Day': 17, 'DOW': None, 'WOM': None},  # St. Patrick's Day
    {'Month': 11, 'Day': 25, 'DOW': '4', 'WOM': None},  # Black Friday
    {'Month': 12, 'Day': 26, 'DOW': None, 'WOM': None},  # Boxing Day
    ]

# Iterate over the public holidays and create the 'public_holiday' column
    df['PUBLIC_HOLIDAY'] = 0  # Initialize the column with 0 (not a public holiday)
    for holiday in public_holidays:
        month_mask = df['date'].dt.month == holiday['Month']
        day_mask = df['date'].dt.day == holiday['Day']
        dow_mask = df['date'].dt.dayofweek == int(holiday['DOW']) if holiday['DOW'] is not None else True
        wom_mask = (df['date'].dt.day - 1) // 7 + 1 == holiday['WOM'] if holiday['WOM'] is not None else True

        mask = month_mask & day_mask & dow_mask & wom_mask
        df.loc[mask, 'PUBLIC_HOLIDAY'] = 1
    return df

In [7]:
algo_table = session.table('ROUTING."ALGO_DATA(With Year)"')
algo_df = algo_table.to_pandas()
df_drop=algo_df.drop("YEAR",axis=1)
final_df=df_drop.drop("SUM(ORDER_TOTAL)",axis=1)
Sales_Forecast_Training_Data_row=session.table('ANALYTICS."Sales_Forecast_Training_Data"')
Sales_Forecast_Training_Data_df = Sales_Forecast_Training_Data_row.to_pandas()
ml_df=final_df[list(Sales_Forecast_Training_Data_df.drop("Profit",axis=1).columns)]

In [8]:
df = pd.DataFrame()

df=ml_df[location_cols].drop_duplicates()

df=generate_date_data(df,datetime_object)


hours = list(range(24))
df['HOUR'] = df.apply(lambda row: hours, axis=1)
df=df.explode('HOUR', ignore_index=True)


In [9]:
trc_df = pd.DataFrame()

trc_df=ml_df[truck_cols].drop_duplicates()

trc_df=generate_date_data(trc_df,datetime_object)

In [10]:
trc_df.drop(["MONTH","DOW","DAY","WOM","YEAR","PUBLIC_HOLIDAY"],axis=1,inplace=True)
merge_df=pd.merge(df, trc_df, how='inner', on="date") 

In [11]:
wdf=session.sql("Select * from ANALYTICS.WEATHER_DATA_API")

wdf=wdf.withColumn("H",F.substring(wdf["TIME"], 12, 2).cast("integer"))

wdf=wdf.withColumn("DATE",F.substring(wdf["TIME"], 0, 10))

wdf=wdf.select("WEATHERCODE","LOCATION_ID","H","DATE" ).to_pandas()

wdf['DATE'] = pd.to_datetime(wdf['DATE'])

wdf.rename(columns = {'H':'HOUR'}, inplace = True)

weadf = pd.merge(wdf, merge_df, right_on=['LOCATION_ID','date',"HOUR"], left_on=['LOCATION_ID','DATE',"HOUR"])
weadf = weadf.drop(['date'], axis=1)
weadf = weadf.drop(['WOM'], axis=1)

In [12]:
latest_date = {'YEAR': 2022.0, 'MONTH': 10.0, 'DAY': 30.0}

# Calculate the date 1 year before the latest date
one_year_before = {'YEAR': latest_date['YEAR'] - 1, 'MONTH': latest_date['MONTH'], 'DAY': latest_date['DAY']}

# Filter the DataFrame to exclude data within the last year
holdout_df_year = algo_df[(algo_df['YEAR'] < one_year_before['YEAR']) | 
                      (algo_df['YEAR'] == one_year_before['YEAR']) & (algo_df['MONTH'] < one_year_before['MONTH']) |
                      (algo_df['YEAR'] == one_year_before['YEAR']) & (algo_df['MONTH'] == one_year_before['MONTH']) & (algo_df['DAY'] <= one_year_before['DAY'])]

X_final_scaled = holdout_df_year.copy()
#Add date column
X_final_scaled.rename(columns={"SUM(ORDER_TOTAL)": "Revenue"},inplace=True)
X_final_scaled['Date'] = pd.to_datetime(X_final_scaled[['YEAR', 'MONTH', 'DAY']])

X_final_scaled.rename(columns = {'Date':'DATE'}, inplace = True)
X_final_scaled_revenue = X_final_scaled.copy()
X_final_scaled = X_final_scaled.drop(['Revenue'], axis=1)
X_final_scaled.info()


df1_columns = set(X_final_scaled.columns)
df2_columns = set(weadf.columns)

columns_only_in_df1 = df1_columns - df2_columns
columns_only_in_df2 = df2_columns - df1_columns

print("Columns only in df1:", columns_only_in_df1)
print("Columns only in df2:", columns_only_in_df2)


merged_df = X_final_scaled.merge(weadf, on=['CITY_BOSTON_ENCODED',
 'CITY_DENVER_ENCODED',
 'CITY_New York City_encoded',
 'CITY_SEATTLE_ENCODED',
 'CITY_San Mateo_encoded',
 'DATE',
 'DAY',
 'DOW',
 'HOUR',
 'LAT',
 'LOCATION_ID',
 'LONG',
 'MENU_TYPE_BBQ_ENCODED',
 'MENU_TYPE_CHINESE_ENCODED',
 'MENU_TYPE_CREPES_ENCODED',
 'MENU_TYPE_ETHIOPIAN_ENCODED',
 'MENU_TYPE_GYROS_ENCODED',
 'MENU_TYPE_Grilled Cheese_encoded',
 'MENU_TYPE_Hot Dogs_encoded',
 'MENU_TYPE_INDIAN_ENCODED',
 'MENU_TYPE_Ice Cream_encoded',
 'MENU_TYPE_Mac & Cheese_encoded',
 'MENU_TYPE_POUTINE_ENCODED',
 'MENU_TYPE_RAMEN_ENCODED',
 'MENU_TYPE_SANDWICHES_ENCODED',
 'MENU_TYPE_TACOS_ENCODED',
 'MENU_TYPE_VEGETARIAN_ENCODED',
 'MONTH',
 'PUBLIC_HOLIDAY',
 'REGION_CA_ENCODED',
 'REGION_CO_ENCODED',
 'REGION_MA_ENCODED',
 'REGION_NY_ENCODED',
 'REGION_WA_ENCODED',
 'TRUCK_ID',
 'WEATHERCODE',
 'YEAR'], how='outer')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242811 entries, 0 to 540200
Data columns (total 39 columns):
 #   Column                                    Non-Null Count   Dtype         
---  ------                                    --------------   -----         
 0   TRUCK_ID                                  242811 non-null  int8          
 1   MONTH                                     242811 non-null  int8          
 2   HOUR                                      242811 non-null  int8          
 3   DOW                                       242811 non-null  int8          
 4   DAY                                       242811 non-null  int8          
 5   PUBLIC_HOLIDAY                            242811 non-null  int8          
 6   LAT                                       242811 non-null  float64       
 7   LONG                                      242811 non-null  float64       
 8   LOCATION_ID                               242811 non-null  int16         
 9   SUM_DAY_OF_WEEK

c:\Users\Nathan\anaconda3\envs\py38_env\lib\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


## Generating Input Data

In [13]:
working_days = 6
truck_ids = [27, 28, 43, 44, 46, 47] 

start_date = datetime.strptime('2021-08-23', '%Y-%m-%d')
dates = [start_date + timedelta(days=i) for i in range(working_days)]

truck_data = []


for i in range(len(truck_ids)):
    num_of_locs = random.randrange(2, 5)
    each_location_travel_distance = random.randrange(8, 12)
    max_total_travel_distance = each_location_travel_distance * num_of_locs

    truck_data.append({
        'Truck_ID': truck_ids[i],
        'Date': dates[i],
        'Starting_Hour': random.randrange(8, 12),
        'Ending_Hour': random.randrange(18, 24),
        'Num_of_locs': num_of_locs,
        'each_location_travel_distance': each_location_travel_distance,
        'Max_Total_Travel_Distance': max_total_travel_distance
    })

truck_df = pd.DataFrame(truck_data)

In [14]:
session.use_schema("RAW_POS")

In [15]:
starting_locations = {}

for truck in truck_ids:
    truck_locations = X_final_scaled[X_final_scaled['TRUCK_ID'] == truck]['LOCATION_ID'].values
    starting_location = truck_locations[0] if len(truck_locations) > 0 else None
    starting_locations[truck] = starting_location

truck_df['Starting_Location'] = truck_df['Truck_ID'].map(starting_locations)

In [16]:
def calculate_list_and_mod(row):
    working_hours = row['working_hour']
    num_locs = row['Num_of_locs']
    each_loc_hours = working_hours // num_locs
    remainder = working_hours % num_locs
    result_list = [each_loc_hours] * num_locs
    for i in range(remainder):
        result_list[i] += 1
    return result_list

In [17]:
def calculate_start_time(row):
    result_list=row["shift_hours"]
    num_locs = row['Num_of_locs']
    start_times = [row['Starting_Hour']]
    for i in range(1, num_locs):
        start_times.append(start_times[-1] + result_list[i - 1])
    
    return start_times

In [18]:
truck_df["working_hour"]=truck_df['Ending_Hour']-truck_df['Starting_Hour']+1
truck_df["shift_hours"]=truck_df.apply(calculate_list_and_mod, axis=1)
truck_df["start_time"]=truck_df.apply(calculate_start_time, axis=1)

In [19]:
truck_df["priority"]=[1,2,3,4,5,6]

In [20]:
truck_df


,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,priority
0,27,2021-08-23,8,23,2,10,20,3304,16,"[8, 8]","[8, 16]",1
1,28,2021-08-24,9,20,3,10,30,3304,12,"[4, 4, 4]","[9, 13, 17]",2
2,43,2021-08-25,10,19,4,8,32,4121,10,"[3, 3, 2, 2]","[10, 13, 16, 18]",3
3,44,2021-08-26,8,23,3,9,27,1917,16,"[6, 5, 5]","[8, 14, 19]",4
4,46,2021-08-27,9,22,3,8,24,15428,14,"[5, 5, 4]","[9, 14, 19]",5
5,47,2021-08-28,9,19,2,10,20,15428,11,"[6, 5]","[9, 15]",6


## Algo

In [21]:
merged_df = merged_df.fillna({ 'SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE':(merged_df['SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE'].mean())})
merged_df = merged_df.fillna({ 'SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE':(merged_df['SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE'].mean())})
predicted_df = merged_df[['TRUCK_ID', 'MONTH', 'HOUR', 'DOW', 'DAY', 'PUBLIC_HOLIDAY', 'LAT',
       'LONG', 'LOCATION_ID', 'SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE',
       'SUM_PREV_YEAR_MONTH_SALES_CITY_MENU_TYPE', 'WEATHERCODE',
       'MENU_TYPE_GYROS_ENCODED', 'MENU_TYPE_CREPES_ENCODED',
       'MENU_TYPE_BBQ_ENCODED', 'MENU_TYPE_SANDWICHES_ENCODED',
       'MENU_TYPE_Mac & Cheese_encoded', 'MENU_TYPE_POUTINE_ENCODED',
       'MENU_TYPE_ETHIOPIAN_ENCODED', 'MENU_TYPE_TACOS_ENCODED',
       'MENU_TYPE_Ice Cream_encoded', 'MENU_TYPE_Hot Dogs_encoded',
       'MENU_TYPE_CHINESE_ENCODED', 'MENU_TYPE_Grilled Cheese_encoded',
       'MENU_TYPE_VEGETARIAN_ENCODED', 'MENU_TYPE_INDIAN_ENCODED',
       'MENU_TYPE_RAMEN_ENCODED', 'CITY_SEATTLE_ENCODED',
       'CITY_DENVER_ENCODED', 'CITY_San Mateo_encoded',
       'CITY_New York City_encoded', 'CITY_BOSTON_ENCODED',
       'REGION_NY_ENCODED', 'REGION_MA_ENCODED', 'REGION_CO_ENCODED',
       'REGION_WA_ENCODED', 'REGION_CA_ENCODED']]
predicted_df

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,LAT,LONG,LOCATION_ID,SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE,...,CITY_SEATTLE_ENCODED,CITY_DENVER_ENCODED,CITY_San Mateo_encoded,CITY_New York City_encoded,CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED
0,17,9,16.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
1,17,9,17.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
2,17,9,18.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
3,17,9,19.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
4,17,9,20.0,4,10,0,39.748742,-104.972158,3304,-0.276074,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625006,3,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,0,1,0,0,0,0,0,0,1
3625007,13,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,0,1,0,0,0,0,0,0,1
3625008,10,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,0,1,0,0,0,0,0,0,1
3625009,5,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,0,1,0,0,0,0,0,0,1


In [22]:
model = joblib.load(open('model.joblib',"rb"))

[17:02:51] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-08de971ced8a8cdc6-1/xgboost/xgboost-ci-windows/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.



In [23]:
predicted_df["HOUR"]=predicted_df["HOUR"].astype(int)
model.predict(predicted_df)

C:\Users\Nathan\AppData\Local\Temp\ipykernel_42116\1277206958.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df["HOUR"]=predicted_df["HOUR"].astype(int)


array([3011.4856, 5846.3574, 7421.3857, ...,  597.321 ,  729.8625,
        592.883 ], dtype=float32)

In [24]:
predictions = model.predict(predicted_df)

In [25]:
predicted_df["predictions"]=predictions

C:\Users\Nathan\AppData\Local\Temp\ipykernel_42116\3702424682.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predicted_df["predictions"]=predictions


In [134]:
unique_location_ids = predicted_df['LOCATION_ID'].unique()

hourly_location_df = pd.DataFrame(columns=[str(hour) for hour in range(1, 25)])

for location_id in unique_location_ids:
    row_data = {str(hour): 0 for hour in range(1, 25)}
    hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)

hourly_location_df['LOCATION_ID'] = unique_location_ids
hourly_location_df = hourly_location_df[['LOCATION_ID'] + [str(hour) for hour in range(1, 25)]]

C:\Users\Nathan\AppData\Local\Temp\ipykernel_42116\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_42116\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_42116\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hourly_location_df = hourly_location_df.append(row_data, ignore_index=True)
C:\Users\Nathan\AppData\Local\Temp\ipykernel_42116\2348948760.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [27]:
hourly_location_df[str(1)]

0       0
1       0
2       0
3       0
4       0
       ..
1873    0
1874    0
1875    0
1876    0
1877    0
Name: 1, Length: 1878, dtype: object

In [112]:
algo_df=predicted_df[["TRUCK_ID","LOCATION_ID","LAT","LONG","predictions","HOUR","MONTH","DOW","DAY"]]

In [29]:
truck_df["current_location"]=None
truck_df["next_start_time"]=None

In [31]:
def calculate_next_start_time(start_times, current_hour):
    if current_hour not in start_times:
        return None
    index = start_times.index(current_hour) + 1
    if index == len(start_times):
        return None
    return start_times[index]

In [48]:
import math

def calculate_distances(df_predictions, starting_location_id):
    def haversine_distance(lat1, lon1, lat2, lon2):
        # Convert latitude and longitude from degrees to radians
        lat1_rad = math.radians(lat1)
        lon1_rad = math.radians(lon1)
        lat2_rad = math.radians(lat2)
        lon2_rad = math.radians(lon2)

        # Haversine formula
        dlon = lon2_rad - lon1_rad
        dlat = lat2_rad - lat1_rad
        a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
        distance = 6371 * c  # Radius of the Earth in kilometers
        return distance

    # Find the reference location based on the starting location ID
    reference_location = df_predictions[df_predictions['LOCATION_ID'] == starting_location_id]
    reference_latitude = reference_location['LAT'].values[0]
    reference_longitude = reference_location['LONG'].values[0]

    # List of other locations with their respective location IDs, latitudes, and longitudes
    other_locations = df_predictions[['LOCATION_ID', 'LAT', 'LONG']].drop_duplicates().values.tolist()

    # Calculate distances between starting location and other locations
    distances = []
    for location in other_locations:
        location_id = location[0]
        latitude = location[1]
        longitude = location[2]
        distance = haversine_distance(reference_latitude, reference_longitude, latitude, longitude)
        distances.append({'Location_ID_start': starting_location_id, 'Location_ID_end': location_id, 'distance': distance})

    # Create a DataFrame from the distances list
    df_distances = pd.DataFrame(distances)

    # Sort the DataFrame by distance in ascending order
    df_distances = df_distances.sort_values('distance')

    return df_distances

In [30]:
truck_df

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,priority,current_location,next_start_time
0,27,2021-08-23,8,23,2,10,20,3304,16,"[8, 8]","[8, 16]",1,None,None
1,28,2021-08-24,9,20,3,10,30,3304,12,"[4, 4, 4]","[9, 13, 17]",2,None,None
2,43,2021-08-25,10,19,4,8,32,4121,10,"[3, 3, 2, 2]","[10, 13, 16, 18]",3,None,None
3,44,2021-08-26,8,23,3,9,27,1917,16,"[6, 5, 5]","[8, 14, 19]",4,None,None
4,46,2021-08-27,9,22,3,8,24,15428,14,"[5, 5, 4]","[9, 14, 19]",5,None,None
5,47,2021-08-28,9,19,2,10,20,15428,11,"[6, 5]","[9, 15]",6,None,None


In [85]:
sorted_df

,TRUCK_ID,LOCATION_ID,LAT,LONG,predictions,HOUR
3126429,47,15504,40.691530,-73.852754,7863.617676,0
620829,47,15212,40.599252,-74.072185,7856.514648,0
478629,47,15514,40.693069,-73.993884,7831.371094,0
2212029,47,15213,40.620892,-74.080391,7826.708008,0
1621629,47,15211,40.614378,-74.084086,7822.833496,0
...,...,...,...,...,...,...
366954,47,3762,42.382125,-71.073457,896.201721,23
919554,47,3731,42.463594,-70.902137,895.138245,23
2991354,47,1368,42.349089,-71.075486,892.253906,23
3450354,47,1285,42.352940,-71.060260,870.841797,23


In [84]:
predicted_df

,TRUCK_ID,MONTH,HOUR,DOW,DAY,PUBLIC_HOLIDAY,LAT,LONG,LOCATION_ID,SUM_DAY_OF_WEEK_AVG_CITY_MENU_TYPE,...,CITY_DENVER_ENCODED,CITY_San Mateo_encoded,CITY_New York City_encoded,CITY_BOSTON_ENCODED,REGION_NY_ENCODED,REGION_MA_ENCODED,REGION_CO_ENCODED,REGION_WA_ENCODED,REGION_CA_ENCODED,predictions
0,17,9,16,4,10,0,39.748742,-104.972158,3304,-0.276074,...,1,0,0,0,0,0,1,0,0,3011.485596
1,17,9,17,4,10,0,39.748742,-104.972158,3304,-0.276074,...,1,0,0,0,0,0,1,0,0,5846.357422
2,17,9,18,4,10,0,39.748742,-104.972158,3304,-0.276074,...,1,0,0,0,0,0,1,0,0,7421.385742
3,17,9,19,4,10,0,39.748742,-104.972158,3304,-0.276074,...,1,0,0,0,0,0,1,0,0,8012.751953
4,17,9,20,4,10,0,39.748742,-104.972158,3304,-0.276074,...,1,0,0,0,0,0,1,0,0,7661.839844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3625006,3,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,1,0,0,0,0,0,0,1,722.258179
3625007,13,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,1,0,0,0,0,0,0,1,585.517395
3625008,10,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,1,0,0,0,0,0,0,1,597.320984
3625009,5,8,23,0,23,0,37.548803,-122.317398,1397,-0.080459,...,0,1,0,0,0,0,0,0,1,729.862488


In [ ]:
for index, truck in truck_df.iterrows():
    predicted_df

In [101]:
table_rows = []

for index, truck in truck_df.iterrows():
    
    algo_df_h = algo_df[(algo_df["HOUR"] == truck['Starting_Hour']) & (algo_df["TRUCK_ID"] == truck['Truck_ID'])]

    filtered_df = algo_df[(algo_df['TRUCK_ID'] == truck['Truck_ID']) & (algo_df["TRUCK_ID"] == truck['Truck_ID'])]

    # Sort the DataFrame by HOUR and Predicted values
    sorted_df = filtered_df.sort_values(['HOUR', 'predictions'], ascending=[True, False])

    # Loop through each hour
    for hour in sorted_df['HOUR'].unique():
    # Filter the DataFrame for the current hour
        hour_df = sorted_df[sorted_df['HOUR'] == hour]

        predicted_values = hour_df['predictions'].tolist()
        best_location_id = hour_df['LOCATION_ID'].iloc[0]

        df_distances = calculate_distances(merged_df, truck['Starting_Location'])
        filtered_distances = df_distances[(df_distances['Location_ID_start'] == truck['Starting_Location']) & (df_distances['distance'] < truck['each_location_travel_distance'])]

        # Get the possible location IDs including the newly added location IDs
        possible_location_ids = sorted(set(hour_df['LOCATION_ID'].unique()) | set(filtered_distances['Location_ID_end'].unique()))

        # Initialize a list to hold the predicted values for each possible location ID
        predicted_values_per_location = []

        # Loop through each possible location ID
        for location_id_end in possible_location_ids:
            # Filter the DataFrame for the current location ID
            location_df = hour_df[hour_df['LOCATION_ID'] == location_id_end]

            # If there are records for the current location ID, append the predicted value
            if not location_df.empty:
                predicted_value = location_df['predictions'].iloc[0]
            else:
                predicted_value = 0  # Default predicted value if no records exist for the current location ID

            predicted_values_per_location.append(predicted_value)

            # Append the row to the table_rows list
        table_rows.append([truck['Truck_ID'] ,hour, best_location_id, possible_location_ids, predicted_values, predicted_values_per_location])

In [109]:
# Create a DataFrame from the table_rows list
df_best_loc_id = pd.DataFrame(table_rows, columns=["Truck", "Hour", "Best Location ID", "Possible Location IDs", "Predicted Values from algo_df", "Predicted Values per Location"])
df_best_loc_id

,Truck,Hour,Best Location ID,Possible Location IDs,Predicted Values from algo_df,Predicted Values per Location
0,27,0,15504,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[6981.1083984375, 6915.62841796875, 6899.36083...","[5312.8784, 5094.419, 5007.3447, 5064.196, 518..."
1,27,1,15504,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[7014.19921875, 6915.62841796875, 6899.3608398...","[5312.8784, 5094.419, 5007.3447, 5064.196, 518..."
2,27,2,15504,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[6981.1083984375, 6915.62841796875, 6899.36083...","[5312.8784, 5094.419, 5007.3447, 5064.196, 518..."
3,27,3,15504,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[6981.1083984375, 6915.62841796875, 6899.36083...","[5312.8784, 5094.419, 5007.3447, 5064.196, 518..."
4,27,4,15504,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[6981.1083984375, 6915.62841796875, 6899.36083...","[5312.8784, 5094.419, 5007.3447, 5064.196, 518..."
...,...,...,...,...,...,...
139,47,19,1282,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[13749.751953125, 13509.779296875, 13469.72558...","[9477.77, 9201.102, 9238.611, 9329.97, 9456.55..."
140,47,20,1282,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[13510.17578125, 13261.56640625, 13255.1757812...","[9100.263, 8995.672, 8960.971, 9050.277, 9202...."
141,47,21,15375,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[12746.744140625, 12481.501953125, 12469.16699...","[7784.368, 7671.6045, 7662.092, 7736.8843, 784..."
142,47,22,5215,"[1030, 1031, 1033, 1034, 1035, 1036, 1037, 103...","[4604.1796875, 4571.8447265625, 4567.895996093...","[1899.3905, 1927.8007, 1861.9933, 2012.0209, 2..."


In [116]:
algo_df.head(5)

,TRUCK_ID,LOCATION_ID,LAT,LONG,predictions,HOUR,MONTH,DOW,DAY
0,17,3304,39.748742,-104.972158,3011.485596,16,9,4,10
1,17,3304,39.748742,-104.972158,5846.357422,17,9,4,10
2,17,3304,39.748742,-104.972158,7421.385742,18,9,4,10
3,17,3304,39.748742,-104.972158,8012.751953,19,9,4,10
4,17,3304,39.748742,-104.972158,7661.839844,20,9,4,10


In [115]:
truck_df

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,priority,current_location,next_start_time
0,27,2021-08-23,8,23,2,10,20,3304,16,"[8, 8]","[8, 16]",1,None,None
1,28,2021-08-24,9,20,3,10,30,3304,12,"[4, 4, 4]","[9, 13, 17]",2,None,None
2,43,2021-08-25,10,19,4,8,32,4121,10,"[3, 3, 2, 2]","[10, 13, 16, 18]",3,None,None
3,44,2021-08-26,8,23,3,9,27,1917,16,"[6, 5, 5]","[8, 14, 19]",4,None,None
4,46,2021-08-27,9,22,3,8,24,15428,14,"[5, 5, 4]","[9, 14, 19]",5,None,None
5,47,2021-08-28,9,19,2,10,20,15428,11,"[6, 5]","[9, 15]",6,None,None


In [139]:
hourly_location_df_save = hourly_location_df.copy()

In [163]:
hourly_location_df = hourly_location_df_save.copy()

In [181]:
# Iterate over each truck
for index, truck in truck_df.iterrows():
    # Extract the shift hours and start times for the truck
    shift_hours = truck["shift_hours"]
    start_times = truck["start_time"]
    next_start_times = truck["next_start_time"]

    # If next_start_times is None, determine next_start_time from start_times list
    if next_start_times is None:
        if start_times and len(start_times) > 1:
            next_start_times = start_times[1:]
        else:
            # If both next_start_times and start_times are empty, use Ending_Hour as fallback
            next_start_times = [truck["Ending_Hour"]]

    best_locations = []

    for i in range(len(shift_hours)):
        shift_hour = shift_hours[i]
        next_start_time = next_start_times[i]
        shift_duration = next_start_time - shift_hour

        algo_df_shift = algo_df[(algo_df["HOUR"] >= shift_hour) & (algo_df["HOUR"] < next_start_time) & (algo_df["TRUCK_ID"] == truck['Truck_ID'])]
        algo_df_shift = algo_df_shift.sort_values(by='predictions', ascending=False)

        best_location = None

        for _, row in algo_df_shift.iterrows():
            location = row["LOCATION_ID"]
            occupied = False

            # Check if the location is available for the entire shift duration
            for hour in range(shift_hour, next_start_time):
                if hourly_location_df.loc[hourly_location_df["LOCATION_ID"] == location, str(hour)].values[0] != 0:
                    occupied = True
                    break

            # If the location is available, assign the truck to the location for the entire shift
            if not occupied:
                best_location = location
                break

        # If no location is available for the entire shift, assign the truck to the next best location
        if not best_location:
            for _, row in algo_df_shift.iterrows():
                location = row["LOCATION_ID"]
                if location in available_locations:
                    best_location = location
                    available_locations.remove(location)
                    break

        if best_location:
            best_locations.append(best_location)
            for hour in range(shift_hour, next_start_time):
                hourly_location_df.loc[hourly_location_df["LOCATION_ID"] == best_location, str(hour)] = truck["Truck_ID"]

    # Update current_location and next_start_time for the truck
    truck_df.at[index, 'current_location'] = best_locations
    if len(next_start_times) > 1:
        truck_df.at[index, 'next_start_time'] = next_start_times[1]
    else:
        truck_df.at[index, 'next_start_time'] = truck["Ending_Hour"]

IndexError: list index out of range

In [145]:
truck_df

,Truck_ID,Date,Starting_Hour,Ending_Hour,Num_of_locs,each_location_travel_distance,Max_Total_Travel_Distance,Starting_Location,working_hour,shift_hours,start_time,priority,current_location,next_start_time
0,27,2021-08-23,8,23,2,10,20,3304,16,"[8, 8]","[8, 16]",1,"[15504.0, 15514.0]",[8]
1,28,2021-08-24,9,20,3,10,30,3304,12,"[4, 4, 4]","[9, 13, 17]",2,"[15504.0, 15514.0, 15513.0]","[4, 4]"
2,43,2021-08-25,10,19,4,8,32,4121,10,"[3, 3, 2, 2]","[10, 13, 16, 18]",3,"[15504.0, 15514.0, 15513.0, 15213.0]","[3, 2, 2]"
3,44,2021-08-26,8,23,3,9,27,1917,16,"[6, 5, 5]","[8, 14, 19]",4,"[15504.0, 15513.0, 15514.0]","[5, 5]"
4,46,2021-08-27,9,22,3,8,24,15428,14,"[5, 5, 4]","[9, 14, 19]",5,"[15504.0, 15514.0, 15513.0]","[5, 4]"
5,47,2021-08-28,9,19,2,10,20,15428,11,"[6, 5]","[9, 15]",6,"[15504.0, 15514.0]",[5]


In [136]:
hourly_location_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1878 entries, 0 to 1877
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   LOCATION_ID  1878 non-null   int16 
 1   1            1878 non-null   object
 2   2            1878 non-null   object
 3   3            1878 non-null   object
 4   4            1878 non-null   object
 5   5            1878 non-null   object
 6   6            1878 non-null   object
 7   7            1878 non-null   object
 8   8            1878 non-null   object
 9   9            1878 non-null   object
 10  10           1878 non-null   object
 11  11           1878 non-null   object
 12  12           1878 non-null   object
 13  13           1878 non-null   object
 14  14           1878 non-null   object
 15  15           1878 non-null   object
 16  16           1878 non-null   object
 17  17           1878 non-null   object
 18  18           1878 non-null   object
 19  19           1878 non-null 

In [50]:
# Iterate over each hour
for hour in range(24):
    
    trucks_starting_now = truck_df[truck_df['start_time'].apply(lambda start_times: hour in start_times)]
    trucks_starting_now.sort_values(by='priority',ascending=True).reset_index(drop=True)
    algo_df_h=algo_df[algo_df["HOUR"]==hour]
    
    truck_df['next_start_time'] = truck_df['start_time'].apply(lambda start_times: calculate_next_start_time(start_times,hour))




    available_locations = []
    for index, truck in trucks_starting_now.iterrows():
     
     
        current_hour = str(hour)
        
        
        for loc_index, loc_row in  hourly_location_df.iterrows():
            if loc_row[current_hour]==0:
                available_locations.append(loc_row["LOCATION_ID"])
            else:
                if loc_row[current_hour]==truck["Truck_ID"]:
                    available_locations.append(loc_row["LOCATION_ID"])
                    
                    hourly_location_df.loc[hourly_location_df[current_hour]==truck["Truck_ID"],current_hour]=0
                    
    availables_locations=list(dict.fromkeys(available_locations))
    
    
    for index, truck in trucks_starting_now.iterrows():
         
         
         ### ditance calculator
         ## remove values from available_locatio based on distance
         ##if u wan any other way also can
        df_distances = calculate_distances(merged_df, truck['Starting_Location']) #calculate distance to all location IDs
        filtered_distances = df_distances[(df_distances['Location_ID_start'] == truck['Starting_Location']) & (df_distances['distance'] < each_location_travel_distance)]
        result = merged_df[merged_df['LOCATION_ID'].isin(filtered_distances['Location_ID_end'])]
         
        algo_df_truck=algo_df_h[algo_df_h["TRUCK_ID"]==truck["Truck_ID"]]
         
        algo_df_loc=algo_df_truck[algo_df_truck["LOCATION_ID"].isin(available_locations)]
         
        if algo_df_loc.size>0:
        
            max_index = algo_df_loc['predictions'].idxmax()
            if pd.notna(max_index):

                row_with_max_value = algo_df_loc.loc[max_index]
                location=row_with_max_value["LOCATION_ID"]
            
        
                df.at[index,'current_location'] = location
                availables_locations.remove(location)

                if pd.notna(truck['next_start_time']) :
            
                    for i in range(hour,int(truck['next_start_time'])):
        
                        column=str(i)
                        hourly_location_df.loc[hourly_location_df["LOCATION_ID"]==location,i]=truck["Truck_ID"]

ValueError: list.remove(x): x not in list

In [ ]:
algo_df_loc=algo_df_truck[algo_df_truck["LOCATION_ID"].isin(available_locations)]

In [ ]:
available_locations.remove(location)

In [ ]:
algo_df_loc=algo_df_truck[algo_df_truck["LOCATION_ID"].isin(available_locations)]
 max_index = algo_df_loc['predictions'].idxmax()
            row_with_max_value = predicted_df.loc[max_index]
            location=row_with_max_value["LOCATION_ID"]
available_locations.remove(location)

2601869    15192
Name: LOCATION_ID, dtype: int16